In [1]:
%%shell
git clone https://github.com/FleshRazer/tmp.git

Cloning into 'tmp'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 24 (delta 0), reused 6 (delta 0), pack-reused 16
Receiving objects: 100% (24/24), 42.35 MiB | 32.98 MiB/s, done.


In [2]:
%%shell
pip3 install --quiet \
    transformers \
    bitsandbytes \
    accelerate \
    datasets \
    peft \
    trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

<font size="5"><b>Restart the runtime after installing accelerate package!</b></font>  

In [2]:
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

from peft import LoraConfig, get_peft_model

from trl import SFTTrainer

from tmp.src.data import data_utils

In [3]:
model_name = "NousResearch/Llama-2-7b-hf"

# Activate 4-bit precision base model loading
use_4bit = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = True

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [4]:
# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

In [5]:
dataset = data_utils.get_filtered_paranmt_hf()

prompt_template = (
    f"Detoxify the text:\n{{reference}}\n---\nDetoxified text:\n{{translation}}{{eos_token}}"
)

def apply_prompt_template(sample):
    return {
        "text": prompt_template.format(
            reference=sample["reference"],
            translation=sample["translation"],
            eos_token=tokenizer.eos_token,
        )
    }

dataset = dataset.map(apply_prompt_template, remove_columns=list(dataset.features))
dataset = dataset.map(lambda x: tokenizer(x["text"]), remove_columns=list(dataset.features), batched=True)
dataset = dataset.train_test_split(test_size=0.05)
dataset


Map:   0%|          | 0/576211 [00:00<?, ? examples/s]

Map:   0%|          | 0/576211 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 547400
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 28811
    })
})

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=2,
        num_train_epochs=1,
        learning_rate=2e-4,
        fp16=False,
        bf16=False,
        logging_steps=25,
        group_by_length=True,
        output_dir="tmp/models/llama2-7b-tmp",
        optim="paged_adamw_8bit",
        report_to="none",
        # report_to="wandb",
        # evaluation_strategy="epoch",
        # preprocess_logits_for_metrics=None,

        # save_strategy="steps",
        # save_steps=25,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    # compute_metrics=compute_metrics,
)

trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,2.303500
50,1.236600
75,1.497700
100,1.178900
125,1.505500
150,1.010800
175,1.527700
200,1.110700
225,1.526300
250,1.133600


In [23]:
%%script false --no-raise-error
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    warmup_steps=2,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    logging_steps=1,
    group_by_length=True,
    output_dir="tmp/models/llama2-7b-tmp",
    optim="paged_adamw_8bit",
    report_to="none",
    # report_to="wandb",
    # evaluation_strategy="epoch",
    # preprocess_logits_for_metrics=None,

    # save_strategy="steps",
    # save_steps=25,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["test"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

trainer.train()